In [ ]:
import importlib
import ufl
import numpy as np
import time
from dolfinx import fem, mesh, io
from mpi4py import MPI
from petsc4py import PETSc
from matplotlib import pyplot as plt
# from util import *
from elasticity.material import Material_el_lin_iso  
from input_files import *
from input_files.Cantilever import Cantilever2D
from elasticity.material import *
from dolfinx import mesh, fem

prob = Cantilever2D(isTest=True)
mat = Material_el_lin_iso()

# importlib.reload(Cantilever2D)

lmd, mu = mat.lmd, mat.mu

def epsilon(_u):
            return ufl.sym(ufl.grad(_u))
def sigma(_u, lmd, mu):
    return 2.0 * mu * epsilon(_u) + lmd * ufl.tr(epsilon(_u)) * ufl.Identity(len(_u))

U1 = fem.VectorFunctionSpace(prob.msh, ("CG", 1)) # displacement basiss
D0 = fem.FunctionSpace(prob.msh, ("DG", 0)) # density
u, v = ufl.TrialFunction(U1), ufl.TestFunction(U1) # note that this is ufl
u_sol = fem.Function(U1) 

k = ufl.inner(sigma(u, lmd, mu), ufl.grad(v)) * ufl.dx




## BC 부과하는 법: 
1. mesh.locate_entities_boundary() 로 boundary facet 찾기
2. fem.locate_dofs_topological() 로 boundary facet 에 해당하는 dofs 찾기
3. fem.dirichletbc() 로 BC 부과하기 

## LC 부과하는 법:
1. mesh.locate_entities() 로 facet 찾기
2. facet_marker 를 만들어서, facet_marker.array() 에 해당하는 부분에 1 부과하기 (NOTE: 전체 no. DOF는 변하지 않음)
3. meshtag 를 이용해서 힘이 부과될 메쉬의 부분 찾기
4. ufl.Measure("ds", domain=prob.msh, subdomain_data=meshtag) 로 measure 만들기
5. ufl.dot(v, prob.LC_force) * ds(1) 로 load form 만들기

## 내부적으로 BC 가 부과되는 법:
- LHS matrix에서 BC 부분을 Identity 로 만들어서, RHS vector 에 BC 부분을 더해준다 (they say lifting)
- 이 때, BC 부분을 Identity 로 만들어주는 법은 _assemble_matrix_mat 참조
- RHS vector 에 BC 부분을 더해주는 법은 fem.petsc.apply_lifting 참조

In [ ]:
ds = ufl.Measure("ds", domain=prob.msh, subdomain_data=prob.LC_facet_tag) 
f = ufl.dot(v, prob.LC_force) * ds(1)
bc_l = fem.dirichletbc(prob.BC_u, prob.BC_dofs, U1)
bcs = [bc_l]

# print(prob.BC_facets)
# print(fem.locate_dofs_topological(U1, prob.dim-1, prob.BC_facets))
# print(bc_l.dof_indices())

# print(prob.LC_facet_tag)

petsc_options={"ksp_type": "preonly","pc_type": "lu","pc_factor_mat_solver_type": "mumps"}
LinProb = fem.petsc.LinearProblem(k, f, bcs=bcs, petsc_options=petsc_options)
u_sol = fem.Function(U1)
u_sol = LinProb.solve()

viewer = PETSc.Viewer(comm=MPI.COMM_WORLD)
A_foo = LinProb._A.copy()
A_foo.zeroEntries()
fem.petsc._assemble_matrix_mat(A_foo, LinProb._a, bcs=[]) 
A_foo.assemble()

# viewer(LinProb._A)
A_foo.view(viewer.ASCII("./savem/testK0.vtu"))
LinProb._A.view(viewer.ASCII("./savem/testK.vtu"))
LinProb._b.view(viewer.ASCII("./savem/testF.vtu"))